# Analysis code for the stellar system evolution

This notebook provides the functions needed to calculate the binned stellar density and velocity anisotropy profiles as well as the Lagrangian radius of the system from simulation outputs. This data is used to create figures 1-3 in the paper "Stellar cores live long and prosper in cuspy dark matter halos." For the raw simulation data, please contact the corresponding author.

Jenni Häkkinen (jenni.hakkinen@helsinki.fi) 2025

## Imports and settings

In [1]:
# Modules
import numpy as np

# If pynbody is not installed, uncomment and run these lines
# import sys
# !{sys.executable} -m pip install --user pynbody
import pynbody as pnb

# Functions
from spherical_coordinates import spherical_components

## Stellar density profile

In [37]:
def stellar_density_profile(file, cut=0.95, nbins=50, particles_per_bin=95):
    """
    Calculate the binned stellar density profile from a GADGET-4 snapshot.

    Parameters
    ----------
    file : str
        GADGET-4 snapshot in .hdf5 format
    cut : float
        Fraction of total particles to be included (default 0.95)
    nbins : int
        Number of bins (default 50)
    particles_per_bin : int
        Number of particles per bin (default 95)

    Returns
    -------
    bin_radii : list
        Mean radius in each bin [kpc]
    densities : list
        Stellar density at the mean radius in each bin [Msol / kpc^3]
    """

    # Load and center snapshot (particle positions)
    f = pnb.load(file)
    f.physical_units('kpc', 'km s^-1', 'Msol')

    com = pnb.analysis.halo.shrink_sphere_center(f.s, min_particles=50)
    f.translate(-com)

    # Pick the inner 95% of particles
    radii = sorted(f.s['r'])
    radii = radii[:int(cut*len(radii))]
    mass = f.s['mass'][0]

    if nbins * particles_per_bin != len(radii):
        raise ValueError('Binning does not cover all particles! Ensure that nbins * particles_per_bin = number of particles.')

    bin_radii = []
    densities = []

    # Bin the particles and calculate densities
    for i in range(nbins):
        start = i * particles_per_bin
        end = start + particles_per_bin

        if end > len(radii):
            break

        # Particles in the current bin
        bin_particles = radii[start:end]

        # Compute volume of the spherical shell
        r_inner = bin_particles[0]
        r_outer = bin_particles[-1]
        volume = (4/3) * np.pi * (r_outer**3 - r_inner**3)

        # Calculate density (mass per volume, mass=1.0002917 per particle)
        density = particles_per_bin * mass / volume if volume > 0 else 0

        # Use the average radius of the bin
        bin_radii.append(np.mean(bin_particles))
        densities.append(density)

    return bin_radii, densities

## Velocity dispersion profile

In [3]:
def velocity_dispersion_profile(file, cut=0.95, nbins=50, particles_per_bin=95, rS=0.2):
    """
    Calculate the velocity dispersion profile from a GADGET-4 snapshot.

    Parameters
    ----------
    file : str
        GADGET-4 snapshot in .hdf5 format
    cut : float
        Fraction of total particles to be included (default 0.95)
    nbins : int
        Number of bins (default 50)
    particles_per_bin : int
        Number of particles per bin (default 95)
    rS : float
        Plummer scale radius (default 0.2) [kpc]

    Returns
    -------
    bin_radii : list
        Mean radius in each bin [kpc]
    sigmas : list
        Velocity dispersion at the mean radius in each bin
    """

    # Load and center snapshot (particle positions and velocities)
    f = pnb.load(file)
    f.physical_units('kpc', 'km s^-1', 'Msol')

    pnb.analysis.halo.center(f.s, mode='ssc', cen_num_particles=95)

    # Pick the inner 95% of particles
    radii = sorted(f.s['r'])
    radii = np.array(radii[:int(cut*len(radii))])

    sort_ind = np.argsort(radii)
    pos = f.s['pos'][sort_ind]
    vel = f.s['vel'][sort_ind]

    bin_radii = []
    sigmas = []

    if nbins * particles_per_bin != len(radii):
        raise ValueError('Binning does not cover all particles! Ensure that nbins * particles_per_bin = number of particles.')

    # Bin the particles and calculate betas
    for i in range(nbins):
        start = i * particles_per_bin
        end = start + particles_per_bin

        if end > len(radii):
            break

        # Particles in the current bin
        pos_bin = pos[start:end]
        vel_bin = vel[start:end]

        # Radial velocity dispersion
        vel_spherical = spherical_components(pos_bin, vel_bin)
        sigma_r, _, _ = np.std(vel_spherical, axis=0)

        # Use the average radius of the bin
        bin_radii.append(np.mean(radii[start:end]))
        sigmas.append(sigma_r)

    return bin_radii, sigmas

## Velocity anisotropy profile

In [ ]:
def beta_parameter(pos, vel):
    """
    Calculate the velocity anisotropy (beta) parameter.
    Returns also the radial velocity dispersion.

    Parameters
    ----------
    pos : list
        Particle positions [kpc]
    vel : list
        Particle velocities

    Returns
    -------
    beta : float
        Velocity anisotropy
    sigma_r : float
        Radial velocity dispersion
    """

    # Velocity dispersions
    vel_spherical = spherical_components(pos, vel)
    sigma_r, sigma_theta, sigma_phi = np.std(vel_spherical, axis=0)

    # Anisotropy parameter
    beta = 1 - (sigma_theta**2 + sigma_phi**2) / (2 * sigma_r**2)

    return beta

In [ ]:
def beta_profile(file, cut=0.95, nbins=50, particles_per_bin=95, rS=0.2):
    """
    Calculate the velocity dispersion and anisotropy (beta) profiles from a GADGET-4 snapshot.

    Parameters
    ----------
    file : str
        GADGET-4 snapshot in .hdf5 format
    cut : float
        Fraction of total particles to be included (default 0.95)
    nbins : int
        Number of bins (default 50)
    particles_per_bin : int
        Number of particles per bin (default 95)
    rS : float
        Plummer scale radius (default 0.2) [kpc]

    Returns
    -------
    bin_radii : list
        Mean radius in each bin [kpc]
    betas : list
        Velocity anisotropy at the mean radius in each bin
    sigmas : list
        Velocity dispersion at the mean radius in each bin
    """

    # Load and center snapshot (particle positions and velocities)
    f = pnb.load(file)
    f.physical_units('kpc', 'km s^-1', 'Msol')

    pnb.analysis.halo.center(f.s, mode='ssc', cen_num_particles=95)

    # Pick the inner 95% of particles
    radii = sorted(f.s['r'])
    radii = np.array(radii[:int(cut*len(radii))])

    sort_ind = np.argsort(radii)
    pos = f.s['pos'][sort_ind]
    vel = f.s['vel'][sort_ind]

    bin_radii = []
    betas = []

    if nbins == 2:
        mask_inner = radii <= rS
        mask_outer = radii > rS

        for mask in (mask_inner, mask_outer):
            pos_bin = pos[mask]
            vel_bin = vel[mask]

            # Anisotropy parameter
            beta = beta_parameter(pos_bin, vel_bin)

            # Use the average radius of the bin (for two bins, this is not used for plotting)
            bin_radii.append(np.mean(radii[mask]))
            betas.append(beta)

    else:
        if nbins * particles_per_bin != len(radii):
            raise ValueError('Binning does not cover all particles! Ensure that nbins * particles_per_bin = number of particles.')

        # Bin the particles and calculate betas
        for i in range(nbins):
            start = i * particles_per_bin
            end = start + particles_per_bin

            if end > len(radii):
                break

            # Particles in the current bin
            pos_bin = pos[start:end]
            vel_bin = vel[start:end]

            # Anisotropy parameter
            beta = beta_parameter(pos_bin, vel_bin)

            # Use the average radius of the bin
            bin_radii.append(np.mean(radii[start:end]))
            betas.append(beta)

    return bin_radii, betas

## Lagrangian radius $r_{35}$

In [ ]:
def lagrangian_radius(file, frac=0.35):
    """
    Calculate the Lagrandian radius of stellar particles from a GADGET-4 snapshot.
    Assumes all particles have the same mass.

    Patameters
    ----------
    file : str
        GADGET-4 snapshot in .hdf5 format
    frac : float
        Fraction of particles for the Lagrangian radius (default 0.35)

    Returns
    -------
    r_lagrangian : float
        Lagrangian radius [kpc]

    """

    # Load snapshot
    f = pnb.load(file)
    f.physical_units('kpc', 'km s^-1', 'Msol')

    # Calculate Lagrangian radius for the wanted fraction
    radii = sorted(f.s['r'])
    r_lagrangian = radii[int(frac*len(radii) - 1)]

    return r_lagrangian